In [1]:
import uuid
import kalshi_python
from kalshi_python.models import *
from pprint import pprint

config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://demo-api.kalshi.co/trade-api/v2'

kalshi_api = kalshi_python.ApiInstance(
    email='mahdouch@bu.edu',
    password='GetHooked9165*',
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
pprint(exchangeStatus)

Exchange status response: 
{'exchange_active': True, 'trading_active': True}


In [2]:
nycTicker = 'HIGHNY'

ausTicker = 'HIGHAUS'

chiTicker = 'HIGHCHI'

miaTicker = 'HIGHMIA'

In [3]:
kalshi_api.get_portfolio_settlements()

{'cursor': 'CgwIg938rgYQ2MbYzQESFUhJR0hDSEktMjRGRUIyNy1CNzUuNQ',
 'settlements': [{'market_result': 'no',
                  'no_count': 5501,
                  'no_total_cost': 43010,
                  'revenue': 550100,
                  'settled_time': '2024-03-23T12:01:12.408495Z',
                  'ticker': 'HIGHMIA-24MAR22-B79.5',
                  'yes_count': 0,
                  'yes_total_cost': 0},
                 {'market_result': 'no',
                  'no_count': 31,
                  'no_total_cost': 2401,
                  'revenue': 3100,
                  'settled_time': '2024-03-23T12:01:05.428938Z',
                  'ticker': 'HIGHAUS-24MAR22-T79',
                  'yes_count': 0,
                  'yes_total_cost': 0},
                 {'market_result': 'no',
                  'no_count': 81,
                  'no_total_cost': 4271,
                  'revenue': 8100,
                  'settled_time': '2024-03-23T12:01:05.428938Z',
                  'ticker': 'H

In [6]:
# Event naming
# TICKER-YYMONTHDD
from datetime import datetime 
today = datetime.now()

def get_event_ticker(ticker):
    event = ticker + '-' + today.strftime('%y%b%d').upper()
    return event

def get_event_markets(event):
    event_details = kalshi_api.get_event(event)
    # extract markets
    markets = event_details.markets
    return markets

def get_exchange__status():
    exchangeStatus = kalshi_api.get_exchange_status()
    return exchangeStatus

def place_orders(prediction, markets):
    # round prediction to int with 0.5 rounding
    prediction = int(round(prediction))
    # for each market, place an order
    def place_quick_order(market, side, action = 'buy', quantity = 1):
        # create a quick order
        orderUuid = str(uuid.uuid4())
        orderResponse = kalshi_api.create_order(
            CreateOrderRequest(
                ticker = market.ticker,
                action = action,
                type ='market',
                count = quantity,
                client_order_id = orderUuid,
                side = side,
            )                            
        )
        return orderResponse, orderUuid

    order_list = []
    for market in markets:
        cap_strike = market.cap_strike
        floor_strike = market.floor_strike
        side = None
        # Check if no floor strike
        if floor_strike == None:
            # If prediction is less than cap_strike, create a quick buy order
            if prediction < cap_strike:
                side = "yes"
                order_response, order_uuid = place_quick_order(market, side = "yes")
            else:
                side = "no"
                order_response, order_uuid = place_quick_order(market, side = "no")
        elif cap_strike == None:
            if prediction > floor_strike:
                side = "yes"
                order_response, order_uuid = place_quick_order(market, side = "yes")
            else:
                side = "no"
                order_response, order_uuid = place_quick_order(market, side = "no")
        else:
            if prediction == cap_strike or prediction == floor_strike:
                side = "yes"
                order_response, order_uuid = place_quick_order(market, side = "yes")
            else:
                side = "no"
                order_response, order_uuid = place_quick_order(market, side = "no")
        order_dict = {
            'market': market,
            'order_response': order_response,
            'order_uuid': order_uuid,
            'quantity': 1,
            'type': 'market',
            'ticker': market.ticker,
            'action': 'buy',
            'side': side           
        }
        order_list.append(order_dict)
    return order_list

nyc_event = get_event_ticker(nycTicker)
nyc_markets = get_event_markets(nyc_event)


In [13]:
# change the last char to a 6
nyc_event = get_event_ticker(nycTicker)[:-1] + '6'
nyc_markets = get_event_markets(nyc_event)

In [6]:
import pandas as pd
from df_utils import *

path = 'history/Austin_history.csv'
city_info = get_city_info()

for city in city_info:
    history_path = city_info[city]['history']
    df = pd.read_csv(history_path)
    df['date_time'] = pd.to_datetime(df['date_time'], format='mixed')
    df.to_csv(history_path, index=False)